## API Data Wrangling Mini Project

This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

Note: Use a .env file and put your key in there and python-dotenv to access it in this notebook.

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a .env-example file in this repository to illusrtate what you need. Copy that to a file called .env and use your own api key in that .env file. Make sure you also have a .gitignore file with a line for .env added to it.

The standard Python gitignore is here you can just copy that.

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the requests package, which can be easily downloaded using pip or conda. You can find the documentation for the package here: http://docs.python-requests.org/en/master/

Finally, apart from the requests package, you are encouraged to not use any third party Python packages, such as pandas, and instead focus on what's available in the Python Standard Library (the collections module might come in handy: https://pymotw.com/3/collections/). Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists. You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

### Import libraries

In [1]:
import os
import requests
from dotenv import load_dotenv

### Get API key

In [2]:
# get API key from .env file
load_dotenv(".env")
API_KEY = os.getenv(key='NASDAQ_API_KEY')
print(API_KEY)

Supp5SHmaL5ydax-JBAi


### Make a request

In [3]:
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?api_key=' + API_KEY

In [4]:
# using parameter to get just the latest row
limit = {'limit': 1}
r = requests.get(url, params=limit)
r.json()

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2020-12-01T14:48:09.907Z',
  'newest_available_date': '2020-12-01',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': 1,
  'transform': None,
  'column_index': None,
  'start_date': '2000-06-07',
  'end_date': '2020-12-01',
  'data': [['2020-12-01',
    112.2,
    112.2,
    111.5,
    112.0,
    None,
    51.0,
    5703.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

### 1) Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017

In [5]:
year_2017 = {'start_date': '2017-01-01', 
             'end_date': '2017-12-31'}
r1 = requests.get(url, params=year_2017)
data_2017 = r1.json()

### 2) Convert the returned JSON object into a Python dictionary

In [7]:
print(type(data_2017))

<class 'dict'>


### 3) Calculate what the highest and lowest opening prices were for the stock in this period

In [60]:
column_names = data_2017['dataset']['column_names']
print(column_names)

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [72]:
data = data_2017['dataset']['data']
print(data[0:2])

[['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None], ['2017-12-28', 51.65, 51.82, 51.43, 51.6, None, 40660.0, 2099024.0, None, None, None]]


In [36]:
def get_open(data:list):
    """
    Function that returns all the opening prices from the list provided
    """
    open_prices = []
    for t in data:
        if isinstance(t[1], float):
            open_prices.append(t[1])
        else:
            open_prices.append(0.)
#     print(len(open_prices))
    return open_prices

opening_prices = get_open(data)

In [58]:
l_open = min(opening_prices)
h_open = max(opening_prices)
print('Highest opening price:\t$ {:.2f}'.format(h_open))
print('Lowest opening price:\t$ {:.2f}'.format(l_open))

Highest opening price:	$ 53.11
Lowest opening price:	$ 0.00


### 4) What was the largest change in any one day (based on High and Low price)?

In [84]:
def largest_change(data:list):
    """
    Calculate the largest change in any one day and return the date
    """
    d = {'date': data[0][0],
         'high': data[0][2],
         'low': data[0][3],
         'change': data[0][2] - data[0][3]}
    for i in data:
        change = i[2] - i[3]
        if change > d['change']:
            d['date'] = i[0]
            d['high'] = i[2]
            d['low'] = i[3]
            d['change'] = change
    return d
l_change = largest_change(data)
print(l_change)

{'date': '2017-05-11', 'high': 46.06, 'low': 43.25, 'change': 2.8100000000000023}


### 5) What was the largest change between any two days?

In [93]:
def largest_two_days(data:list):
    """
    Calculate what was the largest change between any two days (based on closing price)
    """
    d = {'date_1': str,
         'close_1': 0.,
         'date_2': str,
         'close_2': 0.,
         'difference': 0.}
    for i in range(len(data) - 1):
        differ = abs(data[i][4] - data[i+1][4])
        if differ > d['difference']:
            d['date_1'] = data[i][0]
            d['close_1'] = data[i][4]
            d['date_2'] = data[i+1][0]
            d['close_2'] = data[i+1][4]
            d['difference'] = differ
    return d

In [99]:
two_days = largest_two_days(data)
print(two_days)

{'date_1': '2017-08-09', 'close_1': 41.81, 'date_2': '2017-08-08', 'close_2': 44.37, 'difference': 2.559999999999995}


### What was the average daily trading volume during this year?